# License

    Jupyter notebook for accessing CUDA
    Copyright (C) 2018 Andre.Brodtkorb@ifi.uio.no, changed in October by André Brodtkorb

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [1]:
#Lets have matplotlib "inline"
%matplotlib inline

#Import packages we need
import numpy as np
import pycuda.compiler as cuda_compiler
from pycuda.gpuarray import GPUArray
import pycuda.driver as cuda_driver

from matplotlib import pyplot as plt

In [2]:
#Initialize CUDA - must be first call to CUDA!
cuda_driver.init()

In [4]:
#Create CUDA context
cuda_device = cuda_driver.Device(0)
print("Using '{:s}'".format(cuda_device.name()))
print(" => compute capability: {:s}".format(str(cuda_device.compute_capability())))
context = cuda_device.make_context()
free, total = cuda_driver.mem_get_info()
print(" => memory: {:d} / {:d} MB available".format(int(free/(1024*1024)), int(total/(1024*1024))))

Using 'Tesla K80'
 => compute capability: (3, 7)
 => memory: 10031 / 11441 MB available


In [9]:
cuda_kernel = """
__global__ void addVectorKernel(float* c, float* a, float* b) {
    unsigned int i = blockIdx.x*blockDim.x + threadIdx.x;
    c[i] = a[i] + b[i];
}

__global__ void addMatrixKernel(float* c, float* a, float* b, int cols) {
    unsigned int i = blockIdx.x*blockDim.x + threadIdx.x;
    unsigned int j = blockIdx.y*blockDim.y + threadIdx.y;
    
    //Pseudocode
    //c[j][i] = a[j][a] + b[j][i]
    unsigned int k = j*cols + i;
    
    c[k] = a[k] + b[k];
}


//NOTE: We need to implement the function matrixVectorKernel
//c = A*b
__global__ void matrixVectorKernel(float* c, float* A, float* b, int a_rows, int a_cols, sum) {
    //NOTE: Write for loop that computes the vector-vector inner product
    //of row j of A by our vector b, and store the result in 
    //c[j] 
    
    unsigned int j = blockIdx.y*blockDim.y + threadIdx.y;
    for (int i = 0; i<a_cols; ++i) {
        unsigned int k = j*a_cols + i;
        
        sum += A[k] * b[i];
    }
    c[j] = sum;
}
"""
module = cuda_compiler.SourceModule(cuda_kernel)
kernel = module.get_function("addMatrixKernel");

CompileError: nvcc compilation of /tmp/tmp3321w0m6/kernel.cu failed
[command: nvcc --cubin -arch sm_37 -I/home/ubuntu/.local/lib/python3.6/site-packages/pycuda/cuda kernel.cu]
[stderr:
kernel.cu(22): error: identifier "sum" is undefined

kernel.cu(30): error: identifier "sum" is undefined

kernel.cu(32): error: identifier "sum" is undefined

3 errors detected in the compilation of "/tmp/tmpxft_00000796_00000000-6_kernel.cpp1.ii".
]

In [ ]:
#Size of our test
test_size = (16, 24)

#Create test input / output data
a = np.random.random((4,3)).astype(np.float32)
b = np.random.random((3.1)).astype(np.float32)
c = np.empty_like((4.1))

fig = plt.figure(figsize=(12,8))
plt.imshow(1,3,1)
plt.colorbar()
fig.show()

In [ ]:
print(a.shape)

In [6]:
#Upload data to the device
#NOTE: We need to make sure that a=(a_rows, a_columns)
# and that b=(a_colmuns, 1) (column vector)
# and that c=(a_rows, 1)
a_g = GPUArray(a.shape, np.float32)
b_g = GPUArray(b.shape, np.float32)
a_g.set(a)
b_g.set(b)

NameError: name 'a' is not defined

In [7]:
#Allocate output data
c_g = GPUArray(c.shape, np.float32)

NameError: name 'c' is not defined

In [8]:
#a.shape = [rows, columns] => [y, x]
domain_size = (a.shape[1], a.shape[0]) # now domain size is [x, y]

#NOTE: We need to change this so that the grid*block is x = 1, y = number of rows in A
block_size = (8, 4, 1) #These need to be [x, y, z]
grid_size = (domain_size[0] // block_size[0], domain_size[1] // block_size[1], 1)

print("Domain size is " + str(domain_size))
print("Block size is " + str(block_size))
print("Grid size is " + str(grid_size))

NameError: name 'a' is not defined

In [ ]:
#Execute program on device
kernel(c_g, a_g, b_g, np.int32(domain_size[0]), block=block_size, grid=grid_size)

#Copy data from device to host
c_g.get(c)

In [ ]:
#Compute reference using Numpy
c_ref = a + b

#Print result
print("C   = ", c)
print("Ref = ", c_ref)
print("Sad = ", np.sum(np.abs(c - c_ref)))

In [ ]:
fig = plt.figure(figsize=(12,8))
plt.subplot(1,2,1)
plt.imshow(c)
plt.subplot(1,2,2)
plt.imshow(c_ref)